In [1]:
# Standard library imports
import os
import time
import copy
import pickle

# Third-party library imports
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import MinMaxScaler, label_binarize
from sklearn.metrics import accuracy_score

# PyTorch imports
import torch
import torch.nn.functional as F

# Project-specific imports (MoGoNet)
from models.models_GCN import init_model_dict, init_optim
from models.train_test_GCN import (
    gen_trte_adj_mat, train_epoch, test_epoch
)
from models.utils import (
    save_model_dict, one_hot_tensor, cal_sample_weight,
    gen_adj_mat_tensor, gen_test_adj_mat_tensor, cal_adj_mat_parameter
)
from models.utils2 import evaluate_model, count_trainable_parameters


from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
cancer_class_mapping = {
    "AML": 3,
    "BRCA": 5,
    "COAD": 4,
    "GBM": 4,
    "KIRC": 4,
    "LIHC": 4,
    "LUSC": 4,
    "OV": 4,
    "SARC": 4,
    "SKCM": 2
}

cancer_type = "AML"
class_num = cancer_class_mapping[cancer_type]
print(f"{cancer_type} 的类别数是: {class_num}")  # 输出: BRCA 的类别数是: 5

AML 的类别数是: 3


In [3]:
# 参数设置
scenario = 1
nn_type = "later-GCN"
# signal_prop = "high"
# signal_level = "high"
# batch_env = f"s{scenario}-k3-{signal_prop}-{signal_level}"
result_path = f"F:/r-env/大课题/模拟试验1_3/result/{cancer_type}/"
prediction_path = f"F:/r-env/大课题/模拟试验1_3/predictions/{cancer_type}/"
# 创建目录（如果不存在）
os.makedirs(result_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)
# labels = np.loadtxt(f"F:/r-env/中期/方法/模拟试验-模拟数据/情景{scenario}/k-3/label-k3.csv", delimiter=',')
cuda = torch.cuda.is_available()

In [4]:
# View and class settings
view_list = [1, 2, 3]  # List of views (e.g., modalities)
num_view = len(view_list)  # Number of views
num_class = 3  # Number of classes
# Model dimensionality settings
# dim_hvcdn = pow(num_class, num_view)  # Dimension of the H-VCDN
dim_hvcdn = 100 
dim_he_list = [300, 200, 100]  # List of dimensions for hidden embeddings
# Adjacency matrix settings
adj_parameter = 4  # Parameter controlling adjacency matrix generation
# Training and learning rate settings
num_epoch_pretrain = 30  # Number of epochs for pretraining
num_epoch = 500  # Total number of training epochs
lr_e_pretrain = 1e-3  # Learning rate for encoder during pretraining
lr_e = 1e-3  # Learning rate for encoder during main training
lr_c = 5e-2  # Learning rate for classifier during main training
# Testing and evaluation settings
test_interval = 2  # Interval for testing the model (in epochs)

dropout_rate = 0.3


In [13]:
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=2023110400)

In [7]:

# 设置信号比例和信号水平的组合
signal_props = ["1"]
signal_levels = ["1"]
# signal_levels = ["mid"]
# signal_dict = {
#     "high": "高",
#     "mid": "中",
#     "low": "低"
# }
# signal_props = ["high"]
# signal_levels = ["high", "mid"]

# 保存每次实验的运行时间
run_times = []

# 创建一个组合的列表，并循环每个组合
for signal_prop in signal_props:
    for signal_level in signal_levels:
        # 获取中文的信号比例和信号水平
        # ch_sig_prop = signal_dict.get(signal_prop, "未知")
        # ch_sig_level = signal_dict.get(signal_level, "未知")

        # 设置当前的 batch_env
        batch_env = f"s{scenario}-{cancer_type}-k{num_class}"

        # 打印当前的组合
        print(f"开始进行使用{nn_type}对{cancer_type}进行分类的实验")

        metrics = {"Batch": [], "CV": [], "Accuracy": [], "F1 Macro": [], "F1 Micro": [], "F1 Weighted": [],
                   "Precision Macro": [], "Precision Micro": [], "Precision Weighted": [],
                   "Recall Macro": [], "Recall Micro": [], "Recall Weighted": [],
                   "AUC Macro": [], "AUC Micro": [], "AUC Weighted": [],
                   "AUPR Macro": [], "AUPR Micro": [], "AUPR Weighted": [], "Cohen Kappa": [], "Training_time": [], "GPU_memory_allocated":[]}
        
        predictions_data = {
            "batch_num": [],
            "fold_num": [],
            "y_true": [],
            "y_pred": []
        }

        # 动态添加概率字段（根据 num_class）
        for i in range(1, num_class + 1):
            predictions_data[f"proba_{i}"] = []    
        # 记录实验的开始时间
        start_time = time.time()

        # 你的实验循环代码
        for batch_num in range(1, 2):
            print(f"开始第 {batch_num} 次模拟实验")

            # 数据路径
            sim_path = f"F:/r-env/data/benchmark-TCGA"
            # checkpoint_path = f"F:/r-env/中期/方法/模拟试验-模型参数/情景{scenario}/k-3/信号比例-{ch_sig_prop}/信号水平-{ch_sig_level}/sim{batch_num}"

            omics1 = pd.read_csv(f"{sim_path}/{cancer_type}/exp.csv", header = 0)
            omics2 = pd.read_csv(f"{sim_path}/{cancer_type}/meth.csv", header = 0)
            omics3 = pd.read_csv(f"{sim_path}/{cancer_type}/mirna.csv", header = 0)

            omics1 = omics1.iloc[:, 1:].to_numpy() 
            omics2 = omics2.iloc[:, 1:].to_numpy() 
            omics3 = omics3.iloc[:, 1:].to_numpy() 

            df_label = pd.read_csv(f"{sim_path}/{cancer_type}/label.csv")
            labels = df_label['label'].values 

            omics_data = [omics1, omics2, omics3]

            for fold_num, (train_idx_raw, test_idx) in enumerate(kfold.split(omics1, labels)):
                print(f"CV{fold_num + 1}")

                # 将train_idx_raw按3:1划分为train_idx和val_idx
                train_idx, val_idx = train_test_split(
                    train_idx_raw, 
                    test_size=0.25,  # 25%作为验证集，75%作为训练集 (3:1比例)
                    random_state=42,
                    stratify=labels[train_idx_raw]  # 保持标签分布
                )

                
                data_tr_list = []
                data_val_list = []
                data_te_list = []
                data_trval_list = []
                data_trte_list = []

                # 通过循环处理每个omics数据集
                for omic in omics_data:
                    train_X = omic[train_idx]
                    val_X = omic[val_idx]
                    test_X = omic[test_idx]
                    data_tr_list.append(torch.FloatTensor(train_X).cuda() )
                    data_val_list.append(torch.FloatTensor(val_X).cuda() )
                    data_te_list.append(torch.FloatTensor(test_X).cuda() )
                    data_trval_list.append(torch.FloatTensor(np.concatenate((train_X, val_X), axis=0)).cuda() )
                    data_trte_list.append(torch.FloatTensor(np.concatenate((train_X, test_X), axis=0)).cuda() )

                train_y, val_y, test_y = labels[train_idx], labels[val_idx], labels[test_idx]

                # model_filepath = os.path.join(checkpoint_path, f"early-GCN-model{fold_num + 1}")
                # os.makedirs(model_filepath, exist_ok=True)
                # checkpoint = ModelCheckpoint(model_filepath, save_weights_only=True, monitor='val_acc', save_best_only=True, mode='max')

                # View and class settings

                # torch.cuda.empty_cache()

                # 准备数据
                # data_tr_list = [torch.FloatTensor(train_X)]
                # data_trval_list = [torch.FloatTensor(np.concatenate((train_X, val_X), axis=0))]
                # data_trte_list = [torch.FloatTensor(np.concatenate((train_X, test_X), axis=0))]

                if cuda:
                    data_tr_list[0] = data_tr_list[0].cuda()
                    data_trval_list[0] = data_trval_list[0].cuda()
                    data_trte_list[0] = data_trte_list[0].cuda()

                num_tr = data_tr_list[0].shape[0]
                num_trval = data_trval_list[0].shape[0]
                num_trte = data_trte_list[0].shape[0]

                labels_trval = np.concatenate((train_y, val_y))
                labels_trte = np.concatenate((train_y, test_y))

                trval_idx = {
                    "tr": list(range(num_tr)),
                    "te": list(range(num_tr, num_trval))
                }
                
                trte_idx = {
                    "tr": list(range(num_tr)),
                    "te": list(range(num_tr, num_trte))
                }
                
                labels_tr_tensor = torch.LongTensor(labels_trval[trval_idx["tr"]])
                onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)
                sample_weight_tr = torch.FloatTensor(cal_sample_weight(labels_trval[trval_idx["tr"]], num_class))
                
                if cuda:
                    labels_tr_tensor = labels_tr_tensor.cuda()
                    onehot_labels_tr_tensor = onehot_labels_tr_tensor.cuda()
                    sample_weight_tr = sample_weight_tr.cuda()

                dim_list = [x.shape[1] for x in data_tr_list]

                adj_tr_list, adj_val_list = gen_trte_adj_mat(data_tr_list, data_trval_list, trval_idx, adj_parameter)

                # 初始化模型
                model_dict = init_model_dict(num_view, num_class, dim_list, dim_he_list, dim_hvcdn, dropout_rate)
                optim_dict = init_optim(num_view, model_dict, lr_e_pretrain, lr_c)
                
                for model in model_dict.values():
                    if cuda:
                        model.cuda()

                
                # 记录单个fold的开始时间
                fold_start_time = time.time()


                # 预训练
                for epoch in range(num_epoch_pretrain):
                    train_epoch(data_tr_list, adj_tr_list, labels_tr_tensor, 
                                onehot_labels_tr_tensor, sample_weight_tr, model_dict, optim_dict, train_VCDN=True)
                
                # 主训练
                optim_dict = init_optim(num_view, model_dict, lr_e, lr_c)

                # 早停机制
                best_accuracy = 0.0
                patience = 25
                no_improvement_count = 0
                best_model = None


                for epoch in range(num_epoch):
                    train_epoch(data_tr_list,adj_tr_list, labels_tr_tensor, 
                               onehot_labels_tr_tensor, sample_weight_tr, model_dict, optim_dict, train_VCDN=True)
                    
                    # 每10个epoch验证一次
                    if epoch % 2 == 0:
                        val_prob = test_epoch(data_trval_list, adj_val_list,trval_idx["te"], model_dict) ####
                        predictions = np.argmax(val_prob, axis=1)
                        accuracy = accuracy_score(val_y, predictions)
                        
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            no_improvement_count = 0
                            best_model = copy.deepcopy(model_dict)
                        else:
                            no_improvement_count += 2
                            
                        if no_improvement_count >= patience:
                            break
                        # 记录单个fold的结束时间
                # log_gpu_memory()
                fold_end_time = time.time()
                fold_elapsed_time = fold_end_time - fold_start_time

                memory_allocated = torch.cuda.memory_allocated() / 1024**2

                # 最终测试
                adj_tr_list, adj_te_list = gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter)
                predictions = test_epoch(data_trte_list, adj_te_list, trte_idx["te"], best_model)
                y_pred = np.argmax(predictions, axis=1)
                # y_true = labels_trte[trte_idx["te"]]    
                y_true = test_y

                prediction_to_add = {
                    "batch_num": batch_num,
                    "fold_num": fold_num + 1,
                    "y_true": y_true.tolist(),
                    "y_pred": y_pred.tolist()
                }

                # 动态添加概率预测（从 predictions 数组中按列提取）
                for i in range(num_class):
                    prediction_to_add[f"proba_{i+1}"] = predictions[:, i].tolist()

                # 动态添加概率预测（从 predictions 数组中按列提取）
                for key, values in prediction_to_add.items():
                    if key in ["batch_num", "fold_num"]:
                        # 对非列表字段（如batch_num），扩展为与样本数等长的列表
                        predictions_data[key].extend([values] * len(y_true))
                    else:
                        # 对列表字段（如y_true, proba_1等），直接扩展
                        predictions_data[key].extend(values) 
        

                scores = evaluate_model(y_true, y_pred, predictions, class_num)

                for key in metrics:
                    if key == "Batch":
                        metrics[key].append(batch_num)
                    elif key == "CV":
                        metrics[key].append(fold_num + 1)
                    elif key == "Training_time":
                        metrics[key].append(fold_elapsed_time)
                    elif key == "GPU_memory_allocated":
                        metrics[key].append(memory_allocated)  # 记录 GPU 内存占用
                    else:
                        metrics[key].append(scores.get(key, None))  # 没有的指标填None

        end_time = time.time() 

        trainable_params = count_trainable_parameters(model_dict)
        # print(f"模型可训练参数数量: {trainable_params:,}")
        
        metrics_df = pd.DataFrame(metrics)
        float_cols = metrics_df.select_dtypes(include=['float64']).columns
        metrics_df[float_cols] = metrics_df[float_cols].round(6)
        result_filepath = os.path.join(result_path, f"metrics-{nn_type}-{cancer_type}.csv")
        metrics_df.to_csv(result_filepath, index=False)

        predictions_data = pd.DataFrame(predictions_data)
        if not predictions_data.empty:
            # 筛选需要保留4位小数的列
            float_cols = [f'proba_{i+1}' for i in range(num_class)]  # 根据你的实际列名调整
            # 应用四舍五入
            predictions_data[float_cols] = predictions_data[float_cols].round(4)
        predictions_filepath = os.path.join(prediction_path, f"prediction-{nn_type}-{cancer_type}.csv")
        predictions_data.to_csv(predictions_filepath, index=False)
        print(f"当前批次实验完成，结果已保存到: {result_filepath}")


        elapsed_time = end_time - start_time  # 计算每次实验的运行时间
        run_times.append({
            # "signal_prop": signal_prop,
            # "signal_level": signal_level,
            "training_num": batch_num * 4,
            "elapsed_time": elapsed_time,   
            "trainable_params": trainable_params
        })

        
# 将所有运行时间保存到 CSV 文件
run_times_df = pd.DataFrame(run_times)
result_filepath = f"{result_path}/{nn_type}_{cancer_type}_实验运行时间.csv"
run_times_df.to_csv(result_filepath, index=False)

print(f"所有实验完成，运行时间已保存到: {result_filepath}")


开始进行使用later-GCN对AML进行分类的实验
开始第 1 次模拟实验
CV1


KeyboardInterrupt: 

In [15]:
# 设置信号比例和信号水平的组合
signal_props = ["1"]
signal_levels = ["1"]

# 保存每次实验的运行时间

result_path = f"F:/r-env/大课题/模拟试验1_3/result/"
prediction_path = f"F:/r-env/大课题/模拟试验1_3/predictions/"

# 癌症类型和对应类别数的映射
cancer_class_mapping = {
    # "AML": 3,
    "COAD": 4,
    "GBM": 4,
    "KIRC": 4,
    "LIHC": 4,
    "LUSC": 4,
    "OV": 4,
    "SARC": 4,
    "SKCM": 2,
    "BRCA": 5
}

# 遍历所有癌症类型
for cancer_type, class_num in cancer_class_mapping.items():
    num_class = class_num
    print(f"\n开始进行使用{nn_type}对{cancer_type}进行分类的实验，类别数为: {num_class}")
    
    # 创建一个组合的列表，并循环每个组合
    for signal_prop in signal_props:
        for signal_level in signal_levels:
            # 设置当前的 batch_env
            batch_env = f"s{scenario}-{cancer_type}-k{num_class}"

            run_times = []

            metrics = {"Batch": [], "CV": [], "Accuracy": [], "F1 Macro": [], "F1 Micro": [], "F1 Weighted": [],
                      "Precision Macro": [], "Precision Micro": [], "Precision Weighted": [],
                      "Recall Macro": [], "Recall Micro": [], "Recall Weighted": [],
                      "AUC Macro": [], "AUC Micro": [], "AUC Weighted": [],
                      "AUPR Macro": [], "AUPR Micro": [], "AUPR Weighted": [], "Cohen Kappa": [], 
                      "Training_time": [], "GPU_memory_allocated":[]}
            
            predictions_data = {
                "batch_num": [],
                "fold_num": [],
                "y_true": [],
                "y_pred": []
            }

            # 动态添加概率字段（根据 num_class）
            for i in range(1, num_class + 1):
                predictions_data[f"proba_{i}"] = []
            
            # 记录实验的开始时间
            start_time = time.time()

            # 实验循环代码
            for batch_num in range(1, 2):
                print(f"开始第 {batch_num} 次模拟实验")

                # 数据路径
                sim_path = f"F:/r-env/data/benchmark-TCGA"

                omics1 = pd.read_csv(f"{sim_path}/{cancer_type}/exp.csv", header = 0)
                omics2 = pd.read_csv(f"{sim_path}/{cancer_type}/meth.csv", header = 0)
                omics3 = pd.read_csv(f"{sim_path}/{cancer_type}/mirna.csv", header = 0)

                omics1 = omics1.iloc[:, 1:].to_numpy() 
                omics2 = omics2.iloc[:, 1:].to_numpy() 
                omics3 = omics3.iloc[:, 1:].to_numpy() 

                df_label = pd.read_csv(f"{sim_path}/{cancer_type}/label.csv")
                labels = df_label['label'].values 
                
                omics_data = [omics1, omics2, omics3]

                for fold_num, (train_idx_raw, test_idx) in enumerate(kfold.split(omics1, labels)):
                    print(f"CV{fold_num + 1}")

                    # 将train_idx_raw按3:1划分为train_idx和val_idx
                    train_idx, val_idx = train_test_split(
                        train_idx_raw, 
                        test_size=0.20,  # 25%作为验证集，75%作为训练集 (3:1比例)
                        random_state=42,
                        stratify=labels[train_idx_raw]  # 保持标签分布
                    )

                    
                    data_tr_list = []
                    data_val_list = []
                    data_te_list = []
                    data_trval_list = []
                    data_trte_list = []

                    # 通过循环处理每个omics数据集
                    for omic in omics_data:
                        train_X = omic[train_idx]
                        val_X = omic[val_idx]
                        test_X = omic[test_idx]
                        data_tr_list.append(torch.FloatTensor(train_X).cuda() )
                        data_val_list.append(torch.FloatTensor(val_X).cuda() )
                        data_te_list.append(torch.FloatTensor(test_X).cuda() )
                        data_trval_list.append(torch.FloatTensor(np.concatenate((train_X, val_X), axis=0)).cuda() )
                        data_trte_list.append(torch.FloatTensor(np.concatenate((train_X, test_X), axis=0)).cuda() )

                    train_y, val_y, test_y = labels[train_idx], labels[val_idx], labels[test_idx]

                    num_tr = data_tr_list[0].shape[0]
                    num_trval = data_trval_list[0].shape[0]
                    num_trte = data_trte_list[0].shape[0]

                    labels_trval = np.concatenate((train_y, val_y))
                    labels_trte = np.concatenate((train_y, test_y))

                    trval_idx = {
                        "tr": list(range(num_tr)),
                        "te": list(range(num_tr, num_trval))
                    }
                    
                    trte_idx = {
                        "tr": list(range(num_tr)),
                        "te": list(range(num_tr, num_trte))
                    }
                    
                    labels_tr_tensor = torch.LongTensor(labels_trval[trval_idx["tr"]])
                    onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)
                    sample_weight_tr = torch.FloatTensor(cal_sample_weight(labels_trval[trval_idx["tr"]], num_class))
                    
                    if cuda:
                        labels_tr_tensor = labels_tr_tensor.cuda()
                        onehot_labels_tr_tensor = onehot_labels_tr_tensor.cuda()
                        sample_weight_tr = sample_weight_tr.cuda()

                    dim_list = [x.shape[1] for x in data_tr_list]

                    adj_tr_list, adj_val_list = gen_trte_adj_mat(data_tr_list, data_trval_list, trval_idx, adj_parameter)

                    # 初始化模型
                    model_dict = init_model_dict(num_view, num_class, dim_list, dim_he_list, dim_hvcdn, dropout_rate)
                    optim_dict = init_optim(num_view, model_dict, lr_e_pretrain, lr_c)
                    
                    for model in model_dict.values():
                        if cuda:
                            model.cuda()

                    
                    # 记录单个fold的开始时间
                    fold_start_time = time.time()


                    # 预训练
                    for epoch in range(num_epoch_pretrain):
                        train_epoch(data_tr_list, adj_tr_list, labels_tr_tensor, 
                                    onehot_labels_tr_tensor, sample_weight_tr, model_dict, optim_dict, train_VCDN=True)
                    
                    # 主训练
                    optim_dict = init_optim(num_view, model_dict, lr_e, lr_c)

                    # 早停机制
                    best_accuracy = 0.0
                    patience = 25
                    no_improvement_count = 0
                    best_model = None


                    for epoch in range(num_epoch):
                        train_epoch(data_tr_list,adj_tr_list, labels_tr_tensor, 
                                onehot_labels_tr_tensor, sample_weight_tr, model_dict, optim_dict, train_VCDN=True)
                        
                        # 每10个epoch验证一次
                        if epoch % 2 == 0:
                            val_prob = test_epoch(data_trval_list, adj_val_list,trval_idx["te"], model_dict) ####
                            predictions = np.argmax(val_prob, axis=1)
                            accuracy = accuracy_score(val_y, predictions)
                            
                            if accuracy > best_accuracy:
                                best_accuracy = accuracy
                                no_improvement_count = 0
                                best_model = copy.deepcopy(model_dict)
                            else:
                                no_improvement_count += 2
                                
                            if no_improvement_count >= patience:
                                break
                            # 记录单个fold的结束时间
                    # log_gpu_memory()
                    fold_end_time = time.time()
                    fold_elapsed_time = fold_end_time - fold_start_time

                    memory_allocated = torch.cuda.memory_allocated() / 1024**2

                    # 最终测试
                    adj_tr_list, adj_te_list = gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter)
                    predictions = test_epoch(data_trte_list, adj_te_list, trte_idx["te"], best_model)
                    y_pred = np.argmax(predictions, axis=1)
                    # y_true = labels_trte[trte_idx["te"]]    
                    y_true = test_y


                    prediction_to_add = {
                        "batch_num": batch_num,
                        "fold_num": fold_num + 1,
                        "y_true": y_true.tolist(),
                        "y_pred": y_pred.tolist()
                    }

                    # 动态添加概率预测
                    for i in range(num_class):
                        prediction_to_add[f"proba_{i+1}"] = predictions[:, i].tolist()

                    # 动态添加概率预测
                    for key, values in prediction_to_add.items():
                        if key in ["batch_num", "fold_num"]:
                            predictions_data[key].extend([values] * len(y_true))
                        else:
                            predictions_data[key].extend(values)  
            
                    scores = evaluate_model(y_true, y_pred, predictions, num_class)

                    for key in metrics:
                        if key == "Batch":
                            metrics[key].append(batch_num)
                        elif key == "CV":
                            metrics[key].append(fold_num + 1)
                        elif key == "Training_time":
                            metrics[key].append(fold_elapsed_time)
                        elif key == "GPU_memory_allocated":
                            metrics[key].append(memory_allocated)
                        else:
                            metrics[key].append(scores.get(key, None))

            end_time = time.time() 

            trainable_params = count_trainable_parameters(model_dict)
            
            metrics_df = pd.DataFrame(metrics)
            float_cols = metrics_df.select_dtypes(include=['float64']).columns
            metrics_df[float_cols] = metrics_df[float_cols].round(6)
            result_filepath = os.path.join(result_path, f"{cancer_type}/metrics-{nn_type}-{cancer_type}.csv")
            metrics_df.to_csv(result_filepath, index=False)

            predictions_data = pd.DataFrame(predictions_data)
            if not predictions_data.empty:
                float_cols = [f'proba_{i+1}' for i in range(num_class)]
                predictions_data[float_cols] = predictions_data[float_cols].round(4)
            predictions_filepath = os.path.join(prediction_path, f"{cancer_type}/prediction-{nn_type}-{cancer_type}.csv")
            predictions_data.to_csv(predictions_filepath, index=False)
            print(f"当前癌症类型 {cancer_type} 实验完成，结果已保存到: {result_filepath}")

            elapsed_time = end_time - start_time
            run_times.append({
                "cancer_type": cancer_type,
                "num_class": num_class,
                "training_num": batch_num * 4,
                "elapsed_time": elapsed_time,   
                "trainable_params": trainable_params
            })

            # 将所有运行时间保存到 CSV 文件
            run_times_df = pd.DataFrame(run_times)
            result_filepath = f"{result_path}/{cancer_type}/{nn_type}_{cancer_type}_实验运行时间.csv"
            run_times_df.to_csv(result_filepath, index=False)

            print(f"癌症类型{cancer_type}实验完成，运行时间已保存到: {result_filepath}")


开始进行使用later-GCN对COAD进行分类的实验，类别数为: 4
开始第 1 次模拟实验
CV1
CV2
CV3
CV4
当前癌症类型 COAD 实验完成，结果已保存到: F:/r-env/大课题/模拟试验1_3/result/COAD/metrics-later-GCN-COAD.csv
癌症类型COAD实验完成，运行时间已保存到: F:/r-env/大课题/模拟试验1_3/result//COAD/later-GCN_COAD_实验运行时间.csv

开始进行使用later-GCN对GBM进行分类的实验，类别数为: 4
开始第 1 次模拟实验
CV1
CV2
CV3
CV4
当前癌症类型 GBM 实验完成，结果已保存到: F:/r-env/大课题/模拟试验1_3/result/GBM/metrics-later-GCN-GBM.csv
癌症类型GBM实验完成，运行时间已保存到: F:/r-env/大课题/模拟试验1_3/result//GBM/later-GCN_GBM_实验运行时间.csv

开始进行使用later-GCN对KIRC进行分类的实验，类别数为: 4
开始第 1 次模拟实验
CV1
CV2
CV3
CV4
当前癌症类型 KIRC 实验完成，结果已保存到: F:/r-env/大课题/模拟试验1_3/result/KIRC/metrics-later-GCN-KIRC.csv
癌症类型KIRC实验完成，运行时间已保存到: F:/r-env/大课题/模拟试验1_3/result//KIRC/later-GCN_KIRC_实验运行时间.csv

开始进行使用later-GCN对LIHC进行分类的实验，类别数为: 4
开始第 1 次模拟实验
CV1
CV2
CV3
CV4
当前癌症类型 LIHC 实验完成，结果已保存到: F:/r-env/大课题/模拟试验1_3/result/LIHC/metrics-later-GCN-LIHC.csv
癌症类型LIHC实验完成，运行时间已保存到: F:/r-env/大课题/模拟试验1_3/result//LIHC/later-GCN_LIHC_实验运行时间.csv

开始进行使用later-GCN对LUSC进行分类的实验，类别数为: 4
开始第 1 次模拟实验
CV1
CV2
CV3
CV4
当前癌症类型 LUS